In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, classification_report

In [ ]:
data = pd.read_csv('pre-edit1.csv')  # Замените на ваш путь к файлу train.csv
valid = pd.read_csv('../csv_file/valid.csv')  # Замените на ваш путь к файлу test.csv

C:\Users\Darin_btw\AppData\Local\Temp\ipykernel_14040\4154387290.py:2: DtypeWarning: Columns (51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,87,88,89,91,95,96,97,99,131,132,133,135,136,140,141,142,144,145,146,147,148,151,152,155,156,157,158,159,160,161,163,164,165,167,168,169,172,173,174,175,176,177,178,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,203,204,207,208,211,212,213,215,216,239,240,241,243,244,245,459,467,563,567,571,575,579,580,583,611,612,795,799,1031,1032,1033,1034,1035,1036,1037,1039,1040,1047,1055,1063,1064,1071,1072,1073,1074,1175,1176,1177,1178,1179,1180,1181,1182,1277,1278,1281,1282,1447,1448,1449,1450,1451,1452,1453,1454,1653,1654,1656,2193,2194,2195,2196,2199) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv('../csv_file/valid.csv')  # Замените на ваш путь к файлу test.csv


In [ ]:
data.info()
valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14456 entries, 0 to 14455
Columns: 775 entries, client_id to col2663
dtypes: float64(772), int64(3)
memory usage: 85.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3642 entries, 0 to 3641
Columns: 2666 entries, report_date to col2663
dtypes: float64(2329), int64(3), object(334)
memory usage: 74.1+ MB


In [ ]:
# Заполнение пропусков медианой только для числовых столбцов
numeric_columns = data.select_dtypes(include=[np.number]).columns
data[numeric_columns] = data[numeric_columns].fillna(data[numeric_columns].median())

test_numeric_columns = valid.select_dtypes(include=[np.number]).columns
valid[test_numeric_columns] = valid[test_numeric_columns].fillna(valid[test_numeric_columns].median())


In [35]:
threshold = 0.75  # Порог пропусков
missing_ratio = data.isnull().mean()
columns_to_drop = missing_ratio[missing_ratio > threshold].index

In [ ]:
data.drop(columns=columns_to_drop, axis=1)
valid.drop(columns=columns_to_drop, axis=1)

,report_date,client_id,target,col1,col2,col3,col4,col5,col6,col7,...,col2654,col2655,col2656,col2657,col2658,col2659,col2660,col2661,col2662,col2663
0,2023-04-01,2,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,4941670.0,4941670.0,4941670.0,6486500.0,6486500.0,6486500.0,4941670.0,4941670.0,4941670.0,0.256919
1,2023-05-01,4,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,11789500.0,11789500.0,11789500.0,6486500.0,6486500.0,6486500.0,11789500.0,11789500.0,11789500.0,0.267221
2,2023-05-01,9,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7000000.0,7000000.0,7000000.0,6486500.0,6486500.0,6486500.0,7000000.0,7000000.0,7000000.0,0.257217
3,2023-03-01,12,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7737230.5,7603419.5,7687750.0,6486500.0,6486500.0,6486500.0,6841532.0,6600000.0,6733329.5,0.256261
4,2023-03-01,14,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7737230.5,7603419.5,7687750.0,6486500.0,6486500.0,6486500.0,10391000.0,10391000.0,10391000.0,0.257348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3637,2023-04-01,4366,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7737230.5,7603419.5,7687750.0,6486500.0,6486500.0,6486500.0,3500000.0,3500000.0,3500000.0,0.256261
3638,2023-04-01,8298,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7737230.5,7603419.5,7687750.0,6486500.0,6486500.0,6486500.0,6841532.0,6600000.0,6733329.5,0.256261
3639,2023-03-01,835,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7737230.5,7603419.5,7687750.0,6486500.0,6486500.0,6486500.0,6841532.0,6600000.0,6733329.5,0.256919
3640,2023-03-01,8300,0,NaN,0.0,0.0,0.0,NaN,NaN,3689700.0,...,7737230.5,7603419.5,7687750.0,6486500.0,6486500.0,6486500.0,6841532.0,6600000.0,6733329.5,0.257217


In [37]:
X = data.drop(columns=['target', 'client_id'], axis=1)
y = data['target']
X_test = test_data.drop(columns=['client_id', 'report_date', 'client_id'], axis=1)

In [38]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [40]:
y_pred = model.predict_proba(X_val)[:, 1]
roc_auc = roc_auc_score(y_val, y_pred)
print(f"ROC-AUC на валидационных данных: {roc_auc}")

ROC-AUC на валидационных данных: 0.9496882981352814


In [41]:
# Убедиться, что тестовые данные содержат только те же столбцы, что и обучающие
X_test = X_test.reindex(columns=X_train.columns, fill_value=0)

# Предсказание вероятностей для тестовых данных
test_data['score'] = model.predict_proba(X_test)[:, 1]

C:\Users\Darin_btw\AppData\Local\Temp\ipykernel_14040\2269128031.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  test_data['score'] = model.predict_proba(X_test)[:, 1]


In [42]:
test_data[['client_id', 'score']].to_csv('predict.csv', index=False)